In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from datetime import date, timedelta
import matplotlib.dates as mdates

c:\Users\prath\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
url = "historic_data/csv/TCS.NS.csv"    
col_names = ['Date','Open','High','Low','Close','Volume','Adj Close']
stocks = pd.read_csv(url, header=0, names=col_names) 
df = pd.DataFrame(stocks).dropna(how='any', axis=0)
date_split = df['Date'].str.split('-').str
df['Year'], df['Month'], df['Day'] = date_split
df["Volume"] = df["Volume"] / 10000

df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')


#sort by date
df = df.sort_values('Date')

# fix the date 
df.reset_index(inplace=True)
df.set_index("Date", inplace=True)

dates_df = df.copy()
dates_df = dates_df.reset_index()

dates_df['Date'] = pd.to_datetime(dates_df['Date'])


# Store the original dates for plotting the predicitons
org_dates = dates_df['Date']

# convert to ints
dates_df['Date'] = dates_df['Date'].map(mdates.date2num)
dates_df.head()
dates_df.tail()

dates_df.tail()

C:\Users\prath\AppData\Local\Temp\ipykernel_21984\461335973.py:6: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df['Year'], df['Month'], df['Day'] = date_split


,Date,index,Open,High,Low,Close,Volume,Adj Close,Year,Month,Day
5124,19443.0,5133,3106.000000,3135.000000,1524217.0,3152.750000,0.31223,3122.300049,27,03,2023
5125,19444.0,5134,3110.149902,3127.000000,1303530.0,3149.550049,0.31159,3115.899902,28,03,2023
5126,19445.0,5135,3104.100098,3129.000000,1740616.0,3150.000000,0.31389,3138.899902,29,03,2023
5127,19447.0,5136,3152.000000,3189.949951,2382581.0,3213.000000,0.32059,3205.899902,31,03,2023
5128,19450.0,5137,3173.750000,3224.949951,1521830.0,3224.949951,0.32000,3200.000000,03,04,2023


In [3]:
X = dates_df.Date
y = dates_df.Close

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
print(y_test)

4103    2097.949951
4104    2089.500000
4105    2096.000000
4106    2070.000000
4107    2075.000000
           ...     
5124    3152.750000
5125    3149.550049
5126    3150.000000
5127    3213.000000
5128    3224.949951
Name: Close, Length: 1026, dtype: float64


In [4]:
paramters = {
        'C':[0.001,0.01,0.1,1,100,1000],
        'epsilon': [
                    0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10,
                    50, 100, 150, 1000
                ],
        'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5, 8, 40, 100, 1000]
    }

In [5]:
import numpy as np
from datetime import timedelta, date
import plotly.graph_objects as go
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVR

def predict_stock_prices(X_train, y_train, n_days):
    paramters = {
        'C': [0.1,1, 10, 100],
        'gamma': [1,0.1,0.01,0.001],
        'epsilon': [0.1,0.01,0.001,0.0001]
    }

    gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid=paramters,
        cv=10,
        scoring='neg_mean_absolute_error',
        error_score='raise'
    )

    X_train = np.arange(len(X_train)).reshape(-1, 1)
    y_train = np.array(y_train).reshape(-1, 1)

    grid_result = gsc.fit(X_train, y_train.ravel())

    # storing and using best parameters
    # best parameters = min error
    best_param = grid_result.best_params_
    svr_model = SVR(kernel='rbf', C=best_param['C'], epsilon=best_param['epsilon'], gamma=best_param['gamma'])
    svr_model.fit(X_train, y_train.ravel())

    output_days = np.arange(len(X_train), len(X_train)+n_days).reshape(-1, 1)

    dates = []
    current = date.today()
    for i in range(n_days): # creating timeline for future dates
        current += timedelta(days=1)
        dates.append(current)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
                x=dates, 
                y=svr_model.predict(output_days),
                mode='lines+markers',
                name='data'))
    fig.update_layout(
        title="Predicted Close Price of next " + str(n_days) + " days",
        xaxis_title="Date",
        yaxis_title="Closed Price"
    )

    return fig


In [6]:
predict_stock_prices(X_test, y_test, 1)

In [9]:
import numpy as np
from datetime import timedelta, date
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVR

def predict_stock_prices(X_train, y_train):
    paramters = {
        'C': [0.1,1, 10, 100],
        'gamma': [1,0.1,0.01,0.001],
        'epsilon': [0.1,0.01,0.001,0.0001]
    }

    gsc = GridSearchCV(
        estimator=SVR(kernel='rbf'),
        param_grid=paramters,
        cv=10,
        scoring='neg_mean_absolute_error',
        error_score='raise'
    )

    X_train = np.arange(len(X_train)).reshape(-1, 1)
    y_train = np.array(y_train).reshape(-1, 1)

    grid_result = gsc.fit(X_train, y_train.ravel())

    # storing and using best parameters
    # best parameters = min error
    best_param = grid_result.best_params_
    svr_model = SVR(kernel='rbf', C=best_param['C'], epsilon=best_param['epsilon'], gamma=best_param['gamma'])
    svr_model.fit(X_train, y_train.ravel())

    output_days = np.arange(len(X_train), len(X_train)+n_days).reshape(-1, 1)

    predicted_prices = svr_model.predict(output_days)

    return predicted_prices


In [10]:
predict_stock_prices(X_train, y_train)

NameError: name 'n_days' is not defined